## Recommender System - Guide du code et rappel théorique

### Structure et ordre d'utilisation

**Mapping**
unzip.py renvoie les datas
constants.py renvoie vers loaders.py

loaders.py renvoie vers coding1.py, userbased.ipynb, contentbased.ipynb, models.py

models.py renvoie vers evaluators.ipynb, hackaton_make_predictions.ipynb, configs.py

configs.py renvoie vers evaluators.ipynb

user_based.ipynb et content_based.ipynb permettent de comprendre la structure des 2 modèles, mais ceux-ci sont doublés et utilités à partir de models.py


**Workflow**
1. Configurer les paramètres souhaités dans Configs.py
2. Run chaque block dans Evaluations.ipynb dans l'ordre
3. Analyser les resultats obtenus dans mlsmm2156/data/small/evaluations

Voici une description de chaque fichier Python de votre projet et de son rôle dans le système de recommandation de films :

1. constants.py

    Rôle : Centralise toutes les constantes globales utilisées à travers le projet.

    Contenu typique :

        Chemins vers les répertoires de données (DATA_PATH, CONTENT_PATH, EVIDENCE_PATH, EVALUATION_PATH).

        Noms des fichiers de données (par exemple, ITEMS_FILENAME, RATINGS_FILENAME, TMDB_FILENAME, NEW_RATINGS_PENDING_FILENAME, USER_PROFILES_FILENAME).

        Noms des colonnes dans les DataFrames (par exemple, ITEM_ID_COL, USER_ID_COL, RATING_COL, LABEL_COL, GENRES_COL).

        Échelle des ratings (RATINGS_SCALE).

    Interaction : Presque tous les autres fichiers (loaders.py, models.py, content.py, recommender.py, training.py, app.py, merge_new_ratings.py) importent et utilisent la classe Constant (ou une instance C) de ce fichier pour accéder à ces valeurs standardisées. Cela rend le code plus maintenable, car si un nom de colonne ou un chemin change, la modification ne doit être faite qu'à un seul endroit.

2. loaders.py

    Rôle : Responsable du chargement, du nettoyage initial et de la préparation des données brutes à partir des fichiers CSV.

    Fonctionnalités clés :

        load_items(): Charge movies.csv, tmdb_full_features.csv, et links.csv. Il fusionne ces informations pour créer un DataFrame df_items_global enrichi avec les attributs des films (titre, année, genres, informations TMDB comme la popularité, les notes moyennes TMDB, etc.). Il effectue un nettoyage important, notamment sur la colonne des genres pour standardiser le format.

        load_ratings(): Charge ratings.csv pour créer df_ratings_global contenant les évaluations des utilisateurs.

    Interaction :

        Utilise constants.py pour les chemins et les noms de colonnes.

        Est appelé par models.py pour charger df_items_global et df_ratings_global qui sont ensuite utilisés globalement.

        Est appelé directement par app.py au démarrage pour s'assurer que l'application dispose des données les plus récentes.

3. models.py

    Rôle : Définit les différentes classes d'algorithmes de recommandation et charge les données globales nécessaires à leur fonctionnement.

    Contenu typique :

        Chargement initial de df_items_global et df_ratings_global en utilisant loaders.py. Ces DataFrames sont ensuite disponibles pour les autres modules qui importent depuis models.py.

        Définition des classes d'algorithmes de recommandation basées sur surprise.AlgoBase :

            ContentBased: Implémente la logique de recommandation basée sur le contenu. Elle crée des features à partir des attributs des items (genres, année, tags, etc.) et entraîne un modèle de régression par utilisateur.

            UserBased: Wrapper pour surprise.KNNWithMeans (approche collaborative basée sur les utilisateurs).

            SVDAlgo: Wrapper pour surprise.SVD (approche collaborative basée sur la factorisation de matrices).

            Potentiellement d'autres modèles (par exemple, ModelBaseline4 mentionné dans configs.py).

    Interaction :

        Utilise constants.py et loaders.py.

        Fournit les DataFrames globaux (df_items_global, df_ratings_global) à content.py, recommender.py, et app.py.

        Les classes de modèles sont utilisées par training.py pour entraîner les modèles et les sauvegarder.

        Les modèles sauvegardés sont ensuite chargés par recommender.py pour faire des prédictions.

4. content.py

    Rôle : Fournit des fonctions utilitaires pour accéder facilement aux métadonnées des films (titre, genres, année, etc.) stockées dans df_items_global.

    Fonctionnalités clés :

        Fonctions comme get_movie_title(), get_movie_genres(), get_movie_release_year(), get_movie_tmdb_vote_average().

        get_movie_details_list(): Récupère les détails pour une liste d'ID de films.

        get_all_movies_for_selection(): Fournit une liste de films pour les formulaires de sélection.

    Interaction :

        Importe df_items_global depuis models.py (ou le charge directement si l'import échoue).

        Utilise constants.py.

        Est utilisé par app.py pour afficher les informations des films dans l'interface utilisateur.

        Est utilisé par recommender.py pour enrichir les recommandations avec les détails des films.

5. training.py

    Rôle : Script hors ligne pour entraîner les différents modèles de recommandation (SVD, UserBased, ContentBased) sur l'ensemble des données d'évaluation et les sauvegarder sur disque.

    Processus :

        Charge les données d'évaluation (df_ratings_global via models.py ou directement le CSV pour surprise.Dataset).

        Construit un trainset Surprise.

        Instancie les modèles définis dans models.py.

        Entraîne chaque modèle sur le trainset.

        Sauvegarde les modèles entraînés (généralement des fichiers .p) dans le répertoire spécifié par C.DATA_PATH / 'recs'.

    Interaction :

        Utilise constants.py, models.py (pour les classes d'algorithmes et df_ratings_global).

        Utilise la bibliothèque surprise pour la manipulation des datasets et la sauvegarde des modèles (dump).

        Ce script est exécuté manuellement (ou par une tâche planifiée) après la mise à jour du fichier principal des évaluations (par exemple, après l'exécution de merge_new_ratings.py).

6. recommender.py

    Rôle : Gère la logique de génération de recommandations pour un utilisateur donné en utilisant les modèles pré-entraînés.

    Fonctionnalités clés :

        load_model(): Charge un modèle sérialisé (fichier .p) depuis le disque. Met en cache les modèles chargés pour éviter les rechargements redondants.

        get_movies_watched_by_user(): Récupère les films déjà vus par un utilisateur.

        get_top_n_recommendations():

            Charge le modèle spécifié.

            Identifie les films candidats pour la recommandation (tous les films moins ceux déjà vus par l'utilisateur, si spécifié).

            Applique des filtres optionnels (genre, année).

            Prédit les scores pour les films candidats en utilisant le modèle chargé.

            Trie les films par score prédit et retourne le top N.

            Enrichit les recommandations avec les détails des films en utilisant content.py.

    Interaction :

        Utilise constants.py, content.py.

        Importe df_ratings_global depuis models.py (ou le charge) pour get_movies_watched_by_user.

        Utilise surprise.dump pour charger les modèles.

        Est intensivement utilisé par app.py pour obtenir et afficher les recommandations personnalisées.

7. app.py

    Rôle : Constitue l'interface utilisateur web de l'application, construite avec Streamlit. C'est le point d'entrée principal pour l'interaction utilisateur.

    Fonctionnalités clés :

        Navigation : Permet de naviguer entre différentes sections (Tops Généraux, Recommandations Personnalisées, Création de Profil).

        Affichage des Tops : Affiche les films les mieux notés globalement ou par genre, avec filtres (année, genre).

        Création de Profil Utilisateur :

            Permet à un nouvel utilisateur de saisir son nom.

            Présente une sélection de films à noter.

            Sauvegarde le nom et les évaluations de l'utilisateur dans des fichiers CSV temporaires (new_ratings_pending.csv, user_profiles.csv) pour un traitement hors ligne.

        Recommandations Personnalisées (Utilisateurs Existants) :

            Permet à un utilisateur de sélectionner son profil (ID) dans une liste.

            Permet de choisir les types de modèles à utiliser pour les recommandations (User-based, Content-based, SVD).

            Appelle recommender.py pour obtenir les recommandations basées sur les modèles généraux pré-entraînés.

            Affiche les recommandations de manière conviviale, avec les détails des films et les scores prédits.

        Gestion de l'état de session (st.session_state) pour maintenir la page active, l'ID utilisateur courant, les nouvelles évaluations en cours de saisie, etc.

    Interaction :

        Utilise streamlit pour tous les aspects de l'interface graphique.

        Utilise constants.py, content.py, recommender.py.

        Charge df_items_global et df_ratings_global via loaders.py au démarrage pour avoir les données à jour.

        Lit user_profiles.csv pour afficher les noms des utilisateurs dans la liste de sélection.

8. merge_new_ratings.py (Nouveau script hors ligne)

    Rôle : Script hors ligne pour fusionner les nouvelles évaluations (enregistrées par app.py dans new_ratings_pending.csv) avec le fichier principal des évaluations (ratings.csv).

    Processus :

        Lit new_ratings_pending.csv.

        Lit ratings.csv (le fichier principal).

        Concatène les deux DataFrames.

        Gère les doublons (par exemple, en conservant la dernière note pour une paire utilisateur-film).

        Écrase ratings.csv avec les données fusionnées.

        Archive ou supprime new_ratings_pending.csv.

    Interaction :

        Utilise constants.py.

        Modifie directement ratings.csv, qui est ensuite utilisé par loaders.py et training.py.

        Ce script est une étape manuelle (ou automatisée) à exécuter avant de relancer training.py.

9. configs.py

    Rôle : Semble destiné à contenir des configurations pour l'évaluation des modèles (par exemple, quels modèles évaluer, quelles métriques utiliser, paramètres de division des données).

    Contenu typique :

        Liste des modèles à évaluer avec leurs hyperparamètres.

        Listes de métriques (MAE, RMSE, Hit Rate, Novelty).

        Paramètres pour la division des données (taille du jeu de test).

        Paramètres pour l'évaluation Leave-One-Out (valeur de N pour le top-N).

    Interaction :

        Importerait les classes de modèles depuis models.py.

        Serait utilisé par un script d'évaluation (par exemple, evaluator.py mentionné dans le diagramme de projet, qui n'est pas encore implémenté ou fourni).

Flux de Données et Processus Général :

    Préparation Initiale (Hors Ligne) :

        Les données brutes (CSV) sont placées dans les répertoires data/small/content et data/small/evidence.

    Chargement des Données (Automatique au démarrage des scripts/app) :

        loaders.py lit les CSV, les nettoie et les transforme en df_items_global et df_ratings_global.

    Entraînement des Modèles (Hors Ligne) :

        training.py utilise df_ratings_global et les définitions de models.py pour entraîner les modèles SVD, UserBased, et ContentBased.

        Les modèles entraînés sont sauvegardés sous forme de fichiers .p dans data/small/recs.

    Interaction Utilisateur via Streamlit (app.py) :

        L'application charge les données via loaders.py.

        Nouvel Utilisateur :

            L'utilisateur fournit son nom et note des films.

            app.py sauvegarde ces informations dans new_ratings_pending.csv et user_profiles.csv.

        Utilisateur Existant :

            L'utilisateur sélectionne son ID.

            app.py appelle recommender.py.

            recommender.py charge les modèles .p appropriés et génère des recommandations.

            app.py affiche ces recommandations.

    Mise à Jour des Données et Modèles (Processus Hors Ligne Périodique) :

        Étape 1 : Exécuter merge_new_ratings.py pour intégrer les évaluations des nouveaux utilisateurs dans ratings.csv.

        Étape 2 : Exécuter training.py pour ré-entraîner tous les modèles généraux avec le ratings.csv mis à jour.

        Après ces étapes, au prochain redémarrage ou rechargement de app.py, les nouveaux utilisateurs seront disponibles dans la liste de sélection et bénéficieront des recommandations des modèles mis à jour.

### Description des fichiers et leurs fonctions

#### constants.py

Ce code définit une classe de constantes Constant qui centralise tous les chemins d'accès et noms de colonnes utilisés pour manipuler les données. L'objectif est d'automatiser et d'uniformiser les notations pour tous les fichiers.

Cela comprend les chemins vers les fichiers de contenus (movies.csv), d'évidences (ratings.csv), les noms des colonnes importantes (comme userId, movieId, rating), ainsi que l'échelle des notes (de 1 à 5). 

#### loaders.py

Ce code sert à charger les données. 

**load_ratings** et **load_movies** : charger les ratings et les movies sous forme de dataframe 
Si surprise_format=True, elle convertit les données au format requis par la bibliothèque Surprise, qui est utilisée pour implémenter et évaluer des algorithmes de recommandation collaboratifs.
Sinon, elle retourne simplement le fichier des ratings sous forme de DataFrame pandas classique, ce qui est utile pour l'exploration, le prétraitement ou l’analyse descriptive.

**export_evaluation_report** : permet d’enregistrer les résultats d’une évaluation (sous forme de DataFrame) dans un fichier CSV, avec un nom basé sur la date du jour pour éviter d’écraser les anciens rapports. Le fichier est sauvegardé dans le dossier défini par C.EVALUATION_PATH.

#### coding1.py

Ce code effectue l'analyse exploratoire des données de notation dans un système de recommandation. Il commence par charger les jeux de données (notes et films), puis calcule des statistiques descriptives essentielles : nombre de notes, d'utilisateurs, de films, fréquence minimale et maximale de notation par film, valeurs de notes possibles, et nombre de films jamais notés. Ensuite, il visualise la distribution des notations par film pour mettre en évidence la "long-tail property", caractéristique fréquente dans les systèmes de recommandation (peu de films très populaires, beaucoup peu notés). Enfin, il construit une matrice creuse (sparse matrix) des interactions utilisateur-film, utile pour les algorithmes collaboratifs, et en calcule la sparsité, c’est-à-dire le taux d'absences d’interactions.

#### models.py

L’idée de ce fichier est de combiner plusieurs stratégies pour estimer la préférence d’un utilisateur pour un item donné, et de produire des recommandations adaptées. Ce fichier inclut à la fois des modèles simples servant de références, des algorithmes classiques comme la factorisation matricielle (SVD), ainsi qu’un modèle content-based qui exploite des caractéristiques détaillées des items et des méthodes d’apprentissage supervisé pour prédire les notes.

**get_top_n** : cette fonction transforme une liste brute de prédictions en une liste organisée des meilleures recommandations pour chaque utilisateur. Elle prend en entrée un ensemble de prédictions (notes estimées par un modèle pour chaque couple utilisateur-item) et retourne pour chaque utilisateur une liste des items les mieux notés selon ces prédictions, limitée à un nombre n choisi. Pour éviter que des prédictions à valeur identique se retrouvent toujours dans le même ordre, la fonction introduit un mélange aléatoire avant de trier par ordre décroissant, ce qui permet d’avoir un comportement plus équitable. Ce mécanisme est utile pour obtenir une liste finale qui pourra être présentée à l’utilisateur.

**ModelBaseline1** : cette classe prédit toujours la même note fixe (2) pour tous les utilisateurs et items, servant de référence simple

**ModelBaseline2** : cette classe génère des prédictions aléatoires dans la plage des notes possibles, pour simuler un modèle sans apprentissage

**ModelBaseline3** : cette classe prédit la moyenne globale des notes observées, ce qui reflète une tendance générale sans personnalisation

**ModelBaseline4(SVD)** : implémentation d’un algorithme plus avancé, basée sur la factorisation matricielle dite SVD. Cette méthode cherche à représenter chaque utilisateur et chaque item dans un espace latent de faible dimension, de manière à modéliser les interactions sous-jacentes qui expliquent les évaluations. Lors de l’entraînement, ce modèle apprend ces représentations latentes à partir du jeu de données. Sa méthode estimate utilise ces représentations pour prédire la note qu’un utilisateur donnerait à un item, en calculant un produit scalaire pondéré des vecteurs latents.

**ContentBased** : copie du modèle dans le fichier content_based.ipynb -> voir explications là-bas

**UserBased** : copie du modèle dans le fichier user_based.ipynb -> voir explications là-bas

#### user_based.ipynb

//1.Loading Data//
Chargement des données, création du trainset et de l'antitest-set

//2. Explore Surprise's user based algorithm//
Ce code met en œuvre un algorithme user-user avec moyennes centrées (KNNWithMeans) à l’aide de la bibliothèque Surprise. Il permet de visualiser les résultats du modèle via une métrique choisie et d'analyser les variations des hyperparamètres.

Voici les étapes clés :
    Le modèle est entraîné sur l’ensemble d’entraînement trainset.
    Une prédiction est ensuite effectuée pour l'utilisateur 11 et le film 364, illustrant l’utilisation du modèle pour une recommandation spécifique.
    Une prédiction de masse est ensuite générée sur l'anti-test set (toutes les paires utilisateur-film inconnues), et les 30 premières sont affichées avec la note estimée.
    Enfin, la matrice de similarité utilisateur-utilisateur est partiellement affichée pour visualiser comment les utilisateurs sont corrélés entre eux selon l’algorithme.

//3. Implement and explore a customizable user-based algorithm//
Algorithme de Surprise. Le modèle apprend donc une matrice de similarité entre utilisateurs, puis prédit les notes d’un utilisateur pour un film en agissant comme une moyenne pondérée des notes données à ce film par les utilisateurs similaires

**__init__(self, k=3, min_k=1, sim_options={}, kwargs)** : permet d'initialiser les paramètres 

**fit(self, trainset)** : Cette méthode prépare tout ce dont le modèle a besoin pour fonctionner : 
il stocke le trainset fourni par Surprise 
puis construit la matrice utilisateur-film ratings_matrix avec des NaN pour les absences de notes
puis calcule la matrice de similarité entre utilisateurs selon la méthode choisie (msd, jaccard, etc.)
puis calcule les moyennes des notes données par chaque utilisateur (utile comme base de prédiction par défaut ou de correction)

**estimate(self, u, i)** : Cette méthode prédit la note que l'utilisateur u pourrait donner à l’item (film) i :
Si l’utilisateur ou le film est inconnu, retourne NaN.
Utilise la moyenne des notes de l'utilisateur u comme prédiction de base.
Cherche tous les autres utilisateurs ayant noté l’item i.
Calcule la différence entre leur note et leur propre moyenne, pondérée par leur similarité avec u.
Prend les k utilisateurs les plus similaires, et combine leur contribution pour ajuster la prédiction.
Si le nombre de voisins valides est supérieur à min_k, la prédiction est retournée ; sinon on garde la moyenne de u.

**compute_ratings_matrix(self)** : Crée une matrice dense ratings_matrix, de taille (n_users, n_items), initialisée à NaN. Elle est remplie avec les notes connues du trainset. Elle est utilisée pour comparer les utilisateurs entre eux

**compute_similarity_matrix(self)** : Construit la matrice de similarité symétrique entre utilisateurs :
Si la méthode est msd (Mean Squared Difference) : compare les notes communes des utilisateurset applique la formule sim = 1 / (1 + MSD) si le support est suffisant.
Si la méthode est jaccard : calcule la similarité entre les ensembles de films notés (indépendamment des valeurs de note).

Le résultat est une matrice carrée sim de taille (n_users, n_users).

**jaccard_similarity(self, row_i, row_j)** : Cette fonction calcule la similarité de Jaccard entre deux utilisateurs en se basant uniquement sur les films qu’ils ont notés (et non sur la note elle-même)

//4. Compare KNNWithMeans with UserBased//

Ce code compare les prédictions du modèle UserBased implémenté manuellement à celles du modèle KNNWithMeans de Surprise, en utilisant les mêmes paramètres de similarité (msd, k=3, min_k=2). L’objectif est de valider que les deux algorithmes produisent des résultats cohérents, ce qui permet de vérifier la justesse de l’implémentation personnalisée du UserBased. Cela sert donc de test d’équivalence entre une version maison et une version de référence.

//5. Compare MSD and Jaccard//

Ce code compare simplement les prédictions faites par la similarité MSD et la similarité Jaccard, tout deux placé dans le modèle UserBased

#### content_based

//Explore and select content features//

Ce bloc de code sert à extraire et construire des caractéristiques (features) descriptives à partir des données de films pour enrichir le modèle de recommandation. Il est utilisé pour de la recherche mais pas forcément utile.

//Build a Content Based Model//

**class ContentBased**
Ce modèle de recommandation basé sur le contenu apprend une fonction de notation personnalisée pour chaque utilisateur. Il transforme d'abord chaque film en un vecteur de caractéristiques (features) descriptives, puis, pour chaque utilisateur, il entraîne un modèle de régression distinct qui apprend à prédire les notes de cet utilisateur en se basant sur les caractéristiques des films qu'il a déjà évalués.

**__init__(self, features_method, regressor_method)** : C'est le constructeur de la classe. Il initialise l'algorithme en stockant les méthodes choisies pour l'extraction des caractéristiques (features_method) et pour le modèle de régression (regressor_method). Il appelle également create_content_features une première fois pour générer le DataFrame global des caractéristiques des items (films), qui sera utilisé par tous les utilisateurs.

**create_content_features(self, features_methods)** : Cette fonction est le "Content Analyzer". Elle charge les informations sur les items (films) et, en fonction des features_methods spécifiés (ex: "title_length", "Genre_binary", "Tags" avec TF-IDF), elle construit un DataFrame où chaque ligne correspond à un film et chaque colonne à une caractéristique extraite et prétraitée (souvent normalisée).

**fit(self, trainset)** : C'est le "Profile Learner". Cette méthode entraîne le modèle.

    Elle s'assure d'abord que les content_features sont à jour (en les recalculant si nécessaire, bien que dans votre code actuel, elles soient déjà calculées dans __init__).
    Elle initialise un dictionnaire user_profile pour stocker un modèle de régression distinct pour chaque utilisateur.
    Pour chaque utilisateur du trainset :
        Elle récupère les films que l'utilisateur a notés et leurs notes.
        Elle associe ces films à leurs caractéristiques (issues de self.content_features).
        Elle utilise ces caractéristiques comme variables explicatives (X) et les notes de l'utilisateur comme variable cible (y).
        Elle entraîne un modèle de régression (spécifié par self.regressor_method, ex: LinearRegression, RandomForestRegressor) sur ces données spécifiques à l'utilisateur.
        Le modèle entraîné pour cet utilisateur est stocké dans self.user_profile[u]. S'il n'y a pas assez de données ou si une méthode de régression n'est pas spécifiée, le profil peut rester None

**estimate(self, u, i)** : C'est le "Scoring Component". Cette méthode prédit la note qu'un utilisateur u donnerait à un item (film) i. Elle vérifie d'abord si l'utilisateur et l'item sont connus dans le trainset. Si l'user est inconnu (cad si self.user_profile[u] est None, par exemple pour un utilisateur avec peu de notes), elle retourne la note moyenne globale du trainset. Elle récupère les caractéristiques de l'item i à partir de self.content_features. Si l'item n'a pas de caractéristiques (n'est pas dans l'index), elle retourne aussi la moyenne globale.
Elle utilise le modèle de régression personnel de l'utilisateur u (stocké dans self.user_profile[u]) pour prédire une note en se basant sur les caractéristiques de l'item i. Elle retourne cette note prédite

// Tester pour voir 

**test_contentbased_class** : Cette fonction test_contentbased_class permet de tester rapidement une implémentation du modèle de recommandation basé sur le contenu (ContentBased)

#### configs.py

Ce fichier agit comme une interface de configuration centralisée pour automatiser les expériences. Il permet de changer les modèles, méthodes, ou paramètres sans modifier le code principal d’évaluation, ce qui rend le système modulaire et facilement extensible.

#### evaluators.ipynb

\\1. Model Validation Functions//

**load_ratings(surprise_format=False)** : 
Cette fonction charge le fichier de ratings (notes données par les utilisateurs aux films).
Si surprise_format=True, elle formate les données pour qu’elles soient compatibles avec la bibliothèque Surprise (utilisée pour entraîner les algorithmes de recommandation).
Sinon, elle retourne simplement un DataFrame pandas brut.

**generate_split_predictions(algo, ratings_dataset, eval_config)** : 
Cette fonction évalue un modèle à l’aide d’un échantillonnage aléatoire : elle divise les données en un ensemble d'entraînement et un ensemble de test (selon test_size).
Elle entraîne l’algorithme (algo) sur les données d’entraînement, puis prédit les notes sur le testset.
Les prédictions obtenues sont ensuite utilisées pour calculer des métriques de précision comme MAE ou RMSE.

**generate_loo_top_n(algo, ratings_dataset, eval_config)** : 
Cette fonction utilise la méthode du Leave-One-Out (LOO), qui consiste à cacher une note par utilisateur pour tester la pertinence des recommandations.
Après entraînement sur les autres notes, l’algorithme génère des recommandations sur les films non vus.
On extrait les top-N recommandations pour chaque utilisateur et on vérifie si l’item retiré en fait partie (ex. via le hit rate).

**generate_full_top_n(algo, ratings_dataset, eval_config)** :
Ici, l’algorithme est entraîné sur la totalité des données disponibles.
Il prédit ensuite les notes sur tous les films que chaque utilisateur n’a jamais notés (anti-testset).
On en extrait les top-N recommandations pour chaque utilisateur.
Cette approche permet d’évaluer la qualité globale des recommandations, notamment leur originalité (ex : avec la métrique novelty).

**precompute_information()** : 
Cette fonction calcule des informations utiles pour certaines métriques d’évaluation.
Elle compte combien de fois chaque film a été noté, puis classe les films par popularité décroissante.
Elle crée un dictionnaire item_to_rank qui associe à chaque film son rang (1 = le plus populaire).
Ce classement est ensuite utilisé pour évaluer la nouveauté des recommandations (préférer des films moins vus).

**create_evaluation_report(eval_config, sp_ratings, precomputed_dict, available_metrics)** : 
C’est la fonction principale qui orchestre l’évaluation de tous les modèles définis dans EvalConfig.
Elle applique successivement les trois types d’évaluation (split, loo, full) et calcule les métriques correspondantes.
Pour chaque modèle, elle entraîne l’algorithme, génère les prédictions et appelle les fonctions d’évaluation adéquates.
Elle compile tous les résultats dans un DataFrame résumé, prêt à être analysé ou affiché dans un rapport final.

\\2. Evaluation metrics//

**get_hit_rate(anti_testset_top_n, testset)** : 
Cette fonction calcule le hit rate, elle vérifie, pour chaque utilisateur, si le film retiré du jeu de données (et placé dans le testset) apparaît dans ses top-N recommandations (anti_testset_top_n).
Un "hit" (succès) vaut 1 si le film est retrouvé, sinon c’est un "fail" (0).
On calcule ensuite la proportion moyenne de succès sur l’ensemble des utilisateurs.
C’est une mesure simple mais efficace pour évaluer la capacité du système à retrouver des films pertinents.

**get_novelty(anti_testset_top_n, item_to_rank)** : 
Cette fonction mesure la nouveauté des recommandations en utilisant le rang de popularité des films.
Plus un film recommandé est impopulaire (rang élevé), plus il est considéré comme "novel" (nouveau).
Elle parcourt toutes les recommandations faites aux utilisateurs, additionne les rangs des films et calcule une moyenne.
Cette moyenne est ensuite normalisée par le nombre total de films pour donner un score compris entre 0 et 1.
Plus le score est élevé, plus le système propose des contenus originaux et rarement vus par les autres.

\\3. Evaluation workflow//

Ce bloc de code sert à évaluer un ou plusieurs modèles de recommandation définis dans EvalConfig à l’aide de différentes métriques. Il commence par charger les données de notation au format Surprise (sp_ratings) et initialise un dictionnaire vide pour les informations pré-calculées (precomputed_dict). Ensuite, la fonction create_evaluation_report est appelée pour entraîner les modèles et calculer les performances selon les métriques définies (comme MAE, RMSE, hit_rate, novelty). Enfin, les résultats sont affichés à l’écran puis exportés grâce à export_evaluation_report

### 📐 Metriques d'évaluations rappel

🔹 1. MAE (Mean Absolute Error) – Erreur absolue moyenne

    Objectif : mesurer la précision moyenne des prédictions du système, en regardant à quel point les prédictions sont éloignées des notes réelles.

    Formule :
    MAE=1N∑i=1N∣r^i−ri∣
    MAE=N1​i=1∑N​∣r^i​−ri​∣

    où :

        r^ir^i​ = note prédite

        riri​ = note réelle

        NN = nombre total de prédictions

    Interprétation :

        Plus MAE est proche de 0, plus les prédictions sont précises.

        Une MAE de 0.5 signifie que, en moyenne, les prédictions sont à 0.5 point d’écart des vraies notes.

🔹 2. RMSE (Root Mean Squared Error) – Racine carrée de l'erreur quadratique moyenne

    Objectif : mesurer la précision globale, mais en pénalisant davantage les grandes erreurs.

    Formule :
    RMSE=1N∑i=1N(r^i−ri)2
    RMSE=N1​i=1∑N​(r^i​−ri​)2

    ​

    Différence avec MAE :

        RMSE met plus de poids sur les grosses erreurs.

        Exemple : une erreur de 2 compte plus fortement qu’une erreur de 1, car elle est au carré.

🔹 3. Hit Rate – Taux de couverture de l’utilisateur

    Objectif : mesurer si l’élément que l’utilisateur a réellement aimé est présent dans les recommandations du système.

    Utilisé dans : Leave-One-Out (LOO)
    On retire un item que l’utilisateur a noté, puis on génère des recommandations, et on regarde s’il est dedans.

    Formule :
    Hit Rate=Nombre de hitsNombre total de tests
    Hit Rate=Nombre total de testsNombre de hits​

    Exemple :
    Si on fait ça pour 100 utilisateurs, et que dans 75 cas le système a recommandé l’item retiré → Hit Rate = 0.75.

    Intérêt :

        C’est une métrique binaire : est-ce que l’item "test" est dans le top-N recommandations ou non ?

        Plus elle est élevée, mieux le système retrouve les goûts passés des utilisateurs.

🔹 4. Novelty – Nouveauté

    Objectif : évaluer si le système recommande des choses originales, peu connues, plutôt que toujours les mêmes blockbusters.

    Pourquoi c’est important ?
    Un système qui recommande toujours les films les plus populaires est peu utile à long terme. La "nouveauté" incite à la diversité des découvertes.

    Comment c’est mesuré ?
    Souvent par :

        Popularité inverse : plus un film est populaire, moins il est "novel".

        Calcul basé sur le log du nombre de vues :
        Novelty=1∣R∣∑i∈R−log⁡2(p(i))
        Novelty=∣R∣1​i∈R∑​−log2​(p(i))

        où p(i)p(i) est la probabilité d'apparition de l'item (fréquence), et RR l'ensemble des items recommandés.

    Interprétation :

        Une valeur plus élevée = des recommandations moins connues, donc plus "originales".